In [1]:
%matplotlib notebook
import matplotlib
import seaborn as sb
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

# Jupyter Specifics
%matplotlib inline
from IPython.display import display, HTML
from ipywidgets.widgets import interact, interactive, IntSlider, FloatSlider, Layout, ToggleButton, ToggleButtons, fixed
display(HTML("<style>.container { width:100% !important; }</style>"))
style = {'description_width': '100px'}
slider_layout = Layout(width='99%')

from time import time
import pickle as pk

In [2]:
from data import *

Getting data:
getting JHU data...
jhu data selected from 1/22/20 to 10/8/20
expanding JHU data : to new (daily), 7-day rolling (smoothed), reporting glitch (corrected) and combined



number of countries listed in JHU database 189
done with JHU data (covid_ts dictionary keys: confirmed, deaths, recovered).
getting owid data...
owid data selected from 1/23/20 to 10/9/20
expanding OWID data : to new (daily), 7-day rolling (smoothed), reporting glitch (corrected) and combined



number of countries listed in OWID database 212
done with OWID data (covid_owid_ts dictionary see .keys()) .
getting ICU and acute care data icus_2012 and WHO ...
WHO acute file found dictionary acute_who
ICU file found dictionary icus_2012
mapping country names between JHU and OWID and extracting common countries...
getting 2017 contact matrix data from 152 countries ...
152 country contact files found 1 A-M and 2 M-Z
Of 186 in countries_common 145 have contact matrices
4 country contact matrices set equal to that of neighbour to complete cluster country set
                   Afghanistan:Pakistan Kosovo:Serbia Moldova:Romania Norway:Sweden
getting UN all sex age group data for 2020 ...
UN contact files found 1 and 2
Of 186 in countries_common 180 have age structure
Kosovo age structure digitized from CIA World Fact Book Image 2018 to complete cluster country set in trimmed excel file
extracting data sets for common countries both databases...
extracting testing data from OWID databa


completed regularization of testing by pwlf and linear adjustment to confirmed cases (linr).
constructing nonlinear adjustment to confirmed cases based on pwlf testing (nonlin and nonlinr ...
completed nonlinear adjustment to confirmed cases.
Done with data.
---------------------------------


In [3]:
# new module data_config imported by data.py as well as Cluster.py
import data_config
data_config.report_correct = False
from Cluster import *

Constructing common synchronized deaths, case and testing data...
mindeaths 100 mindeathspm 0.5
database JHU report correction False
daysync 23 thresh for deaths 10 mindays 150
No of big common countries is 86
---------------------------------
number of countries in total_deaths) 186
number of countries in big 86
synchronizing and trimming time series to common length...
making cases with nonlinear testing adjustment...
done.
to change the nonlinear correction function, call make_cases_adj_nonlin(K), K=2 by default
----------------------------------------
Finished loading Cluster module
----------------------------------------


In [4]:
print(len(countries_jhu_4_owid),len(countries_jhu_2_owid),len(countries_owid),len(countries_jhu))
print('countries in common: owid format')
print(countries_jhu_2_owid)
print('')
print('owid countries not in common set')
print(set(countries_owid)-set(countries_jhu_2_owid))
print('')
print('countries in common: jhu format')
print(countries_owid_to_jhu)
print('')
print(len(bcountries),'bcountries',bcountries)

187 187 212 189
countries in common: owid format
['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'Brunei', 'Bulgaria', 'Burkina Faso', 'Myanmar', 'Burundi', 'Cape Verde', 'Cambodia', 'Cameroon', 'Central African Republic', 'Chad', 'Chile', 'Colombia', 'Comoros', 'Congo', 'Democratic Republic of Congo', 'Costa Rica', "Cote d'Ivoire", 'Croatia', 'Cuba', 'Cyprus', 'Czech Republic', 'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia', 'Swaziland', 'Ethiopia', 'Fiji', 'Finland', 'France', 'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana', 'Greece', 'Grenada', 'Guatemala', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti', 'Vatican', 'Honduras', 'Hungary', 'Iceland', 'India', 'Ind

In [5]:
cases = [c for c in clusdata_all]
cases

['deaths',
 'cases',
 'cases_lin2020',
 'cases_pwlfit',
 'cases_nonlin',
 'cases_nonlinr']

In [6]:
datasets = ['deaths','cases','cases_lin2020','cases_pwlfit','cases_nonlin']

d_countries = [c for c in clusdata_all['deaths']]
c_countries = [c for c in clusdata_all['cases']]
lc_countries = [c for c in clusdata_all['cases_lin2020']]
pc_countries = [c for c in clusdata_all['cases_pwlfit']]
nc_countries = [c for c in clusdata_all['cases_nonlin']]

countries = d_countries

In [7]:
print(len(d_countries))
print(np.sort(d_countries))

71
['Afghanistan' 'Albania' 'Argentina' 'Armenia' 'Australia' 'Austria'
 'Azerbaijan' 'Bahamas' 'Belarus' 'Belgium' 'Bolivia'
 'Bosnia and Herzegovina' 'Brazil' 'Bulgaria' 'Canada' 'Chile' 'Colombia'
 'Croatia' 'Czech Republic' 'Denmark' 'Dominican Republic' 'Ecuador'
 'Egypt' 'El Salvador' 'Finland' 'France' 'Germany' 'Greece' 'Guatemala'
 'Honduras' 'Hungary' 'India' 'Iran' 'Iraq' 'Ireland' 'Israel' 'Italy'
 'Kazakhstan' 'Kosovo' 'Kuwait' 'Kyrgyzstan' 'Lebanon' 'Luxembourg'
 'Macedonia' 'Mexico' 'Moldova' 'Morocco' 'Netherlands' 'Norway' 'Oman'
 'Panama' 'Peru' 'Philippines' 'Poland' 'Portugal' 'Qatar' 'Romania'
 'Russia' 'Saudi Arabia' 'Serbia' 'Slovenia' 'South Africa' 'Spain'
 'Sweden' 'Switzerland' 'Tunisia' 'Turkey' 'Ukraine'
 'United Arab Emirates' 'United Kingdom' 'United States']


In [8]:
# check that all country sets being used are the same and check time series lengths and starting dates
# 72 countries with Oct 9 finish and with mindeaths=100 and mindays=150 and mindeathspm = 0.5
countrysets = [d_countries,c_countries,lc_countries,pc_countries,nc_countries]
print([len(ccs) for ccs in countrysets])
for ccs1 in countrysets:
    print([ccs1 == ccs2 for ccs2 in countrysets])
print([len(clusdata_all[d1]['United States']) for d1 in datasets])
# print(len(total_deaths_x['dates']),len(total_cases_x['dates']),len(testing_x['dates']),total_deaths_x['dates'][0],total_cases_x['dates'][0],testing_x['dates'][0])

[71, 71, 71, 71, 71]
[True, True, True, True, True]
[True, True, True, True, True]
[True, True, True, True, True]
[True, True, True, True, True]
[True, True, True, True, True]
[155, 155, 155, 155, 155]


In [9]:
covid_owid[0].keys()

odict_keys(['iso_code', 'continent', 'location', 'date', 'total_cases', 'new_cases', 'new_cases_smoothed', 'total_deaths', 'new_deaths', 'new_deaths_smoothed', 'total_cases_per_million', 'new_cases_per_million', 'new_cases_smoothed_per_million', 'total_deaths_per_million', 'new_deaths_per_million', 'new_deaths_smoothed_per_million', 'new_tests', 'total_tests', 'total_tests_per_thousand', 'new_tests_per_thousand', 'new_tests_smoothed', 'new_tests_smoothed_per_thousand', 'tests_per_case', 'positive_rate', 'tests_units', 'stringency_index', 'population', 'population_density', 'median_age', 'aged_65_older', 'aged_70_older', 'gdp_per_capita', 'extreme_poverty', 'cardiovasc_death_rate', 'diabetes_prevalence', 'female_smokers', 'male_smokers', 'handwashing_facilities', 'hospital_beds_per_thousand', 'life_expectancy', 'human_development_index'])

In [10]:
covid_ts.keys()

dict_keys(['confirmed', 'deaths', 'recovered', 'new_deaths', 'new_deaths_smoothed', 'deaths_smoothed', 'new_deaths_corrected', 'new_deaths_corrected_smoothed', 'deaths_corrected_smoothed', 'new_confirmed', 'new_confirmed_smoothed', 'confirmed_smoothed', 'new_confirmed_corrected', 'new_confirmed_corrected_smoothed', 'confirmed_corrected_smoothed', 'confirmed_linr_corrected_smoothed', 'new_confirmed_nonlin_corrected_smoothed', 'confirmed_nonlin_corrected_smoothed', 'new_confirmed_nonlinr_corrected_smoothed', 'confirmed_nonlinr_corrected_smoothed'])

In [11]:
covid_owid_ts.keys()

dict_keys(['confirmed', 'deaths', 'recovered', 'tests', 'stringency', 'population', 'population_density', 'gdp_per_capita', 'new_deaths', 'new_deaths_smoothed', 'deaths_smoothed', 'new_deaths_corrected', 'new_deaths_corrected_smoothed', 'deaths_corrected_smoothed', 'new_confirmed', 'new_confirmed_smoothed', 'confirmed_smoothed', 'new_confirmed_corrected', 'new_confirmed_corrected_smoothed', 'confirmed_corrected_smoothed', 'confirmed_linr_corrected_smoothed', 'new_confirmed_nonlin_corrected_smoothed', 'confirmed_nonlin_corrected_smoothed', 'new_confirmed_nonlinr_corrected_smoothed', 'confirmed_nonlinr_corrected_smoothed'])

# Data save

Execute this section once to produce file `data_all.pk`.

In [12]:
miscnms = ['clusdata_all','cases','datasets','contact_dic','age_group_dic']

In [13]:
deathnms = [x for x in dir() if 'deaths' in x]

In [14]:
casenms = [x for x in dir() if 'cases' in x if not callable(eval(x))]

In [15]:
covidnms = [x for x in dir() if 'covid' in x]

In [16]:
popnms = [x for x in dir() if 'population' in x]

In [17]:
# lccountries is type dict_keys, which can't be pickled
countrynms = [x for x in dir() if 'countr' in x and x is not 'lccountries' and not callable(eval(x))]
countrynms = [x for x in dir() if 'countr' in x and (isinstance(eval(x),dict) or isinstance(eval(x),list) or isinstance(eval(x),tuple))]

In [18]:
allnms = deathnms+casenms+covidnms+countrynms+miscnms
allnms = countrynms + covidnms + miscnms + deathnms + casenms + popnms
data_all = {nm:eval(nm) for nm in allnms}

In [19]:
start = time()
pk.dump(data_all,open('./pks/data_all_raw.pk','wb'))
print('elapsed: ',time()-start)

elapsed:  7.242347002029419


# Data Load

Use this code to read in the data, e.g. at the top of another notebook, as an alternative to loading data.py or Cluster.py

In [20]:
# read in data
start=time()
print('reading in data...')
with open('./pks/data_all_raw.pk','rb') as fp:
    foo = pk.load(fp)
print('elapsed: ',time()-start)

# make each element of the dictionary a global variable named with key:
for x in foo:
    stmp = x+"= foo['"+x+"']"
    exec(stmp)

reading in data...
elapsed:  5.852925062179565
